En este ejemplo vamos a ver cómo podemos utilizar un dataset real de datos inmobiliarios para predecir el precio de una vivienda.

Para ello, seguiremos el tutorial que hay en https://www.pyimagesearch.com/2019/01/21/regression-with-keras/.

In [1]:
# nos descargamos los datos
!git clone https://github.com/emanhamed/Houses-dataset
!ls -la Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2166
Receiving objects: 100% (2166/2166), 176.26 MiB | 12.96 MiB/s, done.
Resolving deltas: 100% (20/20), done.
total 92
drwxr-xr-x 4 root root  4096 Jun 18 17:59  .
drwxr-xr-x 1 root root  4096 Jun 18 17:58  ..
drwxr-xr-x 8 root root  4096 Jun 18 17:59  .git
-rw-r--r-- 1 root root   378 Jun 18 17:59  .gitattributes
-rw-r--r-- 1 root root  1726 Jun 18 17:59  .gitignore
drwxr-xr-x 2 root root 69632 Jun 18 17:59 'Houses Dataset'
-rw-r--r-- 1 root root  1767 Jun 18 17:59  README.md


In [8]:
!ls -la Houses-dataset/Houses\ Dataset/

total 186548
drwxr-xr-x 2 root root  69632 Jun 18 17:59 .
drwxr-xr-x 4 root root   4096 Jun 18 17:59 ..
-rw-r--r-- 1 root root 428033 Jun 18 17:59 100_bathroom.jpg
-rw-r--r-- 1 root root  42411 Jun 18 17:59 100_bedroom.jpg
-rw-r--r-- 1 root root 282545 Jun 18 17:59 100_frontal.jpg
-rw-r--r-- 1 root root 223147 Jun 18 17:59 100_kitchen.jpg
-rw-r--r-- 1 root root 259472 Jun 18 17:59 101_bathroom.jpg
-rw-r--r-- 1 root root 276635 Jun 18 17:59 101_bedroom.jpg
-rw-r--r-- 1 root root 234326 Jun 18 17:59 101_frontal.jpg
-rw-r--r-- 1 root root 315347 Jun 18 17:59 101_kitchen.jpg
-rw-r--r-- 1 root root 307371 Jun 18 17:59 102_bathroom.jpg
-rw-r--r-- 1 root root 263332 Jun 18 17:59 102_bedroom.jpg
-rw-r--r-- 1 root root 291815 Jun 18 17:59 102_frontal.jpg
-rw-r--r-- 1 root root 234550 Jun 18 17:59 102_kitchen.jpg
-rw-r--r-- 1 root root 184084 Jun 18 17:59 103_bathroom.jpg
-rw-r--r-- 1 root root 161703 Jun 18 17:59 103_bedroom.jpg
-rw-r--r-- 1 root root 424171 Jun 18 17:59 103_frontal.jpg
-rw-r--

In [11]:
!cat Houses-dataset/Houses\ Dataset/HousesInfo.txt | more

4 4 4053 85255 869500
4 3 3343 36372 865200
3 4 3923 85266 889000
5 5 4022 85262 910000
3 4 4116 85266 971226
4 5 4581 85266 1249000
3 4 2544 85262 799000
4 5 5524 85266 1698000
3 4 4229 85255 1749000
4 5 3550 85262 1500000
5 5 4829 85266 519200
4 4 3428 85255 1039000
5 3 5462 85266 799000
4 4 4021 85266 889000
5 5 4406 85266 700000
4 4 3721 85255 500000
5 3 3710 85331 740000
3 4 2748 85255 725000
5 4 4190 85255 1199000
3 3.5 4143 85266 925000
3 3.5 4229 85255 1749000
6 6.5 5963 85262 1595000
3 3 2685 85255 799900


In [12]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

def load_house_attributes(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
 
    # determine (1) the unique zip codes and (2) the number of data
	# points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()
	# loop over each of the unique zip codes and their corresponding
	# count
	for (zipcode, count) in zip(zipcodes, counts):
		# the zip code counts for our housing dataset is *extremely*
		# unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less
		# than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)
	# return the data frame
	return df

def process_house_attributes(df, train, test):
	# initialize the column names of the continuous data
	continuous = ["bedrooms", "bathrooms", "area"]
	# performin min-max scaling each continuous feature column to
	# the range [0, 1]
	cs = MinMaxScaler()
	trainContinuous = cs.fit_transform(train[continuous])
	testContinuous = cs.transform(test[continuous])
 
    # one-hot encode the zip code categorical data (by definition of
	# one-hot encoing, all output features are now in the range [0, 1])
	zipBinarizer = LabelBinarizer().fit(df["zipcode"])
	trainCategorical = zipBinarizer.transform(train["zipcode"])
	testCategorical = zipBinarizer.transform(test["zipcode"])
	# construct our training and testing data points by concatenating
	# the categorical features with the continuous features
	trainX = np.hstack([trainCategorical, trainContinuous])
	testX = np.hstack([testCategorical, testContinuous])
	# return the concatenated training and testing data
	return (trainX, testX)

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Ahora definiremos el modelo tal y como lo hacen en el tutorial:

<img src="https://www.pyimagesearch.com/wp-content/uploads/2019/01/keras_regression_arch.png">

In [14]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input


def create_mlp(dim, regress=False):
	# define our MLP network
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu"))
	model.add(Dense(4, activation="relu"))
	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))
	# return our model
	return model

In [15]:
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import locale
import os

# construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
print("[INFO] loading house attributes...")
# inputPath = os.path.sep.join([args["dataset"], "HousesInfo.txt"])
inputPath = "Houses-dataset/Houses Dataset/HousesInfo.txt"
df = load_house_attributes(inputPath)
# construct a training and testing split with 75% of the data used
# for training and the remaining 25% for evaluation
print("[INFO] constructing training/testing split...")
(train, test) = train_test_split(df, test_size=0.25, random_state=42)

[INFO] loading house attributes...
[INFO] constructing training/testing split...


In [16]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (this will lead to
# better training and convergence)
maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice

In [17]:
# process the house attributes data by performing min-max scaling
# on continuous features, one-hot encoding on categorical features,
# and then finally concatenating them together
print("[INFO] processing data...")
(trainX, testX) = process_house_attributes(df, train, test)

[INFO] processing data...


In [18]:
# create our MLP and then compile the model using mean absolute
# percentage error as our loss, implying that we seek to minimize
# the absolute percentage difference between our price *predictions*
# and the *actual prices*
model = create_mlp(trainX.shape[1], regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(x=trainX, y=trainY, 
	validation_data=(testX, testY),
	epochs=200, batch_size=8)

[INFO] training model...
Epoch 1/200
34/34 [==============================] - 0s 6ms/step - loss: 164.0460 - val_loss: 115.7033
Epoch 2/200
34/34 [==============================] - 0s 2ms/step - loss: 89.7141 - val_loss: 76.3067
Epoch 3/200
34/34 [==============================] - 0s 2ms/step - loss: 62.1811 - val_loss: 57.2083
Epoch 4/200
34/34 [==============================] - 0s 2ms/step - loss: 51.9385 - val_loss: 56.6125
Epoch 5/200
34/34 [==============================] - 0s 2ms/step - loss: 46.1286 - val_loss: 51.2713
Epoch 6/200
34/34 [==============================] - 0s 2ms/step - loss: 41.5434 - val_loss: 49.9857
Epoch 7/200
34/34 [==============================] - 0s 2ms/step - loss: 40.3948 - val_loss: 46.2468
Epoch 8/200
34/34 [==============================] - 0s 2ms/step - loss: 39.8705 - val_loss: 49.9456
Epoch 9/200
34/34 [==============================] - 0s 2ms/step - loss: 39.2518 - val_loss: 43.0294
Epoch 10/200
34/34 [==============================] - 0s 2ms/ste

In [19]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testX)
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 22.70%, std: 19.53%


Aquí tenéis el ejemplo que combina datos numéricos/categóricos e imágenes: https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/.

Basaos en ese para hacer la práctica! ;)